# Snow Pit Data Ingest
---
## Raw Data Ingest

In [1]:
import xmltodict
import pandas as pd
pd.options.display.max_columns = None

# create dict from xml
# file_path = 'pit_obs.xml'
file_path = "../raw_data/sp/all-MT.xml"
with open(file_path, "rb") as file:
    pit_data = xmltodict.parse(file)

# create dataframe from dict
pit_obs = pd.DataFrame.from_dict(pit_data['Pit_Data']['Pit_Observation'])
pit_obs.columns = [col.replace('@', '') for col in pit_obs.columns]
print(pit_obs.shape)
pit_obs.sample(3)

(500, 52)


,activities,iLayerNumber,iDepth,bld,nid,stability,aviPit,crownObs,aviLoc,incline,winDir,skiAreaPit,bcPit,testPit,windspeed,aspect,sky,windLoading,heightOfSnowpack,precip,serial,version,pitNotes,timestamp,measureFrom,depthUnits,coordType,elvUnits,longType,latType,longitude,lat,zone,north,east,rhoUnits,fractureCat,hardnessScaling,state,range,share,User,Location,Layer,Shear_Test_Result,Density_Profile,Temperature_Profile,bootPen,skiPen,surfGrainType,surfGrainSize,tempUnits
79,,,,7.77,43630,,false,false,,5,,false,,false,,90,,,134,NO,drupal-nid-43630-1652468430000,7.77-0.1,,1648232100000,bottom,cm,lat_long,m,W,N,-110.93384,45.836,,,,kg_cubic_m,shear_quality,linear,MT,Bridger Range,true,"{'@measureFrom': 'bottom', '@depthUnits': 'cm'...","{'@ID': '', '@type': 'LATLONG', '@zone': '', '...","[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...",NaN,{'@profile': '134.0_336?124.0_352?120.0_344?11...,NaN,NaN,NaN,NaN,NaN,C
305,We skied slope;,5,59,7.77,37446,,false,false,,15,SW,false,,false,Light Breeze,250,FEW,no,111,NO,drupal-nid-37446-1641768147000,7.77-0.1,,1641753900000,bottom,cm,lat_long,ft,W,N,-113.91893,48.09398,,,,kg_cubic_m,shear_quality,exponential,MT,Swan Range,true,"{'@measureFrom': 'bottom', '@depthUnits': 'cm'...","{'@ID': '', '@type': 'LATLONG', '@zone': '', '...","[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...","[{'@code': 'CT', '@sdepth': '85.0', '@depthUni...","{'@profile': '', '@depthUnits': 'cm', '@densit...",NaN,NaN,NaN,NaN,NaN,C
417,,8,20,7.77,35450,,false,false,,26,,false,,false,Calm,135,OVC,no,78,S1,drupal-nid-35450-1639102367000,7.77-0.1,,1639079100000,bottom,cm,lat_long,ft,E,N,-113.66795,48.27251,,,,kg_cubic_m,fracture_character,exponential,MT,Flathead Range,true,"{'@measureFrom': 'bottom', '@depthUnits': 'cm'...","{'@ID': '', '@type': 'LATLONG', '@zone': '', '...","[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...","[{'@code': 'ECT', '@sdepth': '56.0', '@depthUn...","{'@profile': '', '@depthUnits': 'cm', '@densit...",NaN,45,NaN,NaN,NaN,C


---
## Flatten Columns

User, Location and Density_Profile can be flattened into columns and concat'd to df

In [2]:
cols_to_flatten = ['User', 'Location', 'Density_Profile']
pit_obs_2 = pit_obs.copy()

for ctf in cols_to_flatten:
    df = pd.DataFrame(pit_obs[ctf].tolist())
    df.columns = [col.replace('@', f'{ctf}_') for col in df.columns]
    pit_obs_2 = pd.concat([pit_obs_2, df], axis='columns')

pit_obs_2.drop(columns=cols_to_flatten, inplace=True)
pit_obs_2.set_index('nid', inplace=True)
pit_obs_2.sample(3)


,activities,iLayerNumber,iDepth,bld,stability,aviPit,crownObs,aviLoc,incline,winDir,skiAreaPit,bcPit,testPit,windspeed,aspect,sky,windLoading,heightOfSnowpack,precip,serial,version,pitNotes,timestamp,measureFrom,depthUnits,coordType,elvUnits,longType,latType,longitude,lat,zone,north,east,rhoUnits,fractureCat,hardnessScaling,state,range,share,Layer,Shear_Test_Result,Temperature_Profile,bootPen,skiPen,surfGrainType,surfGrainSize,tempUnits,User_measureFrom,User_depthUnits,User_tempUnits,User_coordType,User_elvUnits,User_longType,User_latType,User_longitude,User_lat,User_zone,User_north,User_east,User_rhoUnits,User_fractureCat,User_hardnessScaling,User_state,User_range,User_share,User_username,User_useSymbols,User_first,User_last,User_name,User_phone,User_prof,User_email,User_affil,Location_ID,Location_type,Location_zone,Location_ns,Location_ew,Location_east,Location_north,Location_lat,Location_longitude,Location_elv,Location_state,Location_range,Location_name,Density_Profile_profile,Density_Profile_densityUnits,Density_Profile_depthUnits
nid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
41699,Snowmobile tracks on slope; We snowmobiled slo...,4,110,7.77,Good,false,false,,22,,false,,false,Calm,260,BKN,no,135,NO,drupal-nid-41699-1646955487000,7.77-0.1,,1646935200000,bottom,cm,lat_long,ft,W,N,-111.268312,44.933258,,,,kg_cubic_m,shear_quality,linear,MT,Madison Range-S,true,"[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...","{'@code': 'ECT', '@sdepth': '125.0', '@depthUn...",NaN,20,NaN,NaN,NaN,C,bottom,cm,C,lat_long,ft,W,N,-111.268312,44.933258,,,,kg_cubic_m,shear_quality,linear,MT,Madison Range-S,true,ianhoyer,true,Ian,Hoyer,Ian Hoyer,5039840722,true,ianhoyer@gmail.com,Gallatin National Forest Avalanche Center,,LATLONG,,N,W,,,44.933258,-111.268312,9200,MT,Madison Range-S,Cabin Creek,,kg_cubic_m,cm
39680,,,,7.77,,false,false,,19,W,false,,false,Strong,90,OVC,yes,177,NO,drupal-nid-39680-1644349396000,7.77-0.1,,1644093000000,bottom,cm,UTM,m,W,N,,,12T,5009687,465271,kg_cubic_m,fracture_character,exponential,MT,Gallatin Range-N,true,"[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...",NaN,NaN,10,NaN,MFsl,0.3,C,bottom,cm,C,UTM,m,W,N,,,12T,5009687,465271,kg_cubic_m,fracture_character,exponential,MT,Gallatin Range-N,true,nata.deleeuw,true,Nata,de Leeuw,Nata de Leeuw,403-627-4891,true,nata.deleeuw@gmail.com,Yellowstone Club Ski Patrol,,UTM,12T,N,W,465271,5009687,,,2670,MT,Gallatin Range-N,Spirit - 2022-02-05,,kg_cubic_m,cm
40047,We skied slope;,,,7.77,,false,false,,25,,false,,false,Calm,240,CLR,,160,NO,drupal-nid-40047-1644804361000,7.77-0.1,Dug to the ground in a second pass through the...,1644773640000,bottom,cm,lat_long,ft,W,N,-111.3597900,44.9766200,12T,,,kg_cubic_m,shear_quality,linear,MT,Madison Range-S,true,"[{'@grainSizeUnits1': 'mm', '@grainSizeUnits2'...","[{'@code': 'ECT', '@sdepth': '', '@depthUnits'...",NaN,40,NaN,NaN,NaN,C,bottom,cm,C,lat_long,ft,W,N,-111.3597900,44.9766200,12T,,,kg_cubic_m,shear_quality,linear,MT,Madison Range-S,true,hahamarshall,true,Hannah,Marshall,Hannah Marshall,6033919598,false,ehannahmarshall@gmail.com,Gallatin National Forest Avalanche Center,,LATLONG,12T,N,W,,,44.9766200,-111.3597900,8760,MT,Madison Range-S,Wilderness Boundary,,kg_cubic_m,cm


In [3]:
pit_obs_2.User_username.value_counts()

amarienthal      58
ianhoyer         41
davidzinn        34
hahamarshall     30
nata.deleeuw     29
                 ..
Nelsonvt          1
sallyrose         1
PaulHeffernan     1
Seberia8          1
aschoening        1
Name: User_username, Length: 93, dtype: int64

---
## Split Layer and Shear_Test_Result into separate dataframes

In [4]:
layers = pd.DataFrame()

for pit_index, layer_list in pit_obs_2.Layer.items():
    if isinstance(layer_list, dict): # in case pit has only 1 layer, which will be a dict instead of a list
        layer_list = [layer_list]
    df = pd.DataFrame(layer_list)
    df['nid'] = pit_index
    layers = pd.concat([layers, df], axis='rows')

layers.columns = [col.replace('@', '') for col in layers.columns]
layers.head(10)

ValueError: DataFrame constructor not properly called!

In [5]:
shear_test_results = pd.DataFrame()

for pit_index, shear_test_result_list in pit_obs_2.Shear_Test_Result.items():
    if isinstance(shear_test_result_list, dict): # in case pit has only 1 shear test, which will be a dict instead of a list
        shear_test_result_list = [shear_test_result_list]
    if isinstance(shear_test_result_list, list):
        df = pd.DataFrame(shear_test_result_list)
        df['nid'] = pit_index
        shear_test_results = pd.concat([shear_test_results, df], axis='rows')

shear_test_results.columns = [col.replace('@', '') for col in shear_test_results.columns]
shear_test_results.sample(3)

# are character and fractureCharacter the same?

,code,sdepth,score,ecScore,ctScore,quality,dateString,numberOfTaps,fractureCat,fractureCharacter,s,lengthOfCut,lengthOfColumn,releaseType,dataCode,nid,depthUnits
1,ECT,150.0,ECTN,25,,,01/13/2022,25,shear_quality,,ECTN25 150.0 01/13/2022.11:30:00:0,,,,,37881,cm
0,CT,30.0,CT,,21,Q1,11/27/2022,21,shear_quality,,CT21 Q1 30.0 11/27/2022.08:00:00:0,,,,,44971,cm
1,ECT,35.0,ECTN,30,,,01/04/2022,30,shear_quality,,ECTN30 35.0 01/04/2022.12:15:00:0,,,,,37151,cm
